In [46]:
"""
Script to populate database with first 500 properties from NWMLS API
"""
import sys
import os
import requests
from typing import List, Dict, Any

import os
from pathlib import Path

# try:
#     base_dir = Path(__file__).resolve().parent
# except NameError:
base_dir = Path(os.getcwd())

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(base_dir))))

from database.models import init_database, get_session, Property, PropertyMedia
from services.property_transformer import transform_property, transform_media

import argparse
    
parser = argparse.ArgumentParser(description="Populate database with NWMLS properties")
parser.add_argument("--database", default="sqlite:///properties.db", help="Database URL")
parser.add_argument("--limit", type=int, default=500, help="Limit number of properties to insert")

# API Configuration
BEARER_TOKEN = "0ab805914c87f009210f3444be95e11b19e7cf6f"
API_URL = "https://api-demo.mlsgrid.com/v2/Property?$filter=OriginatingSystemName%20eq%20'nwmls'%20and%20MlgCanView%20eq%20true&$expand=Media&$top=500"

headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

def fetch_properties_from_api() -> List[Dict[str, Any]]:
    """Fetch properties from NWMLS API"""
    print("Fetching properties from NWMLS API...")
    response = requests.get(API_URL, headers=headers)
    
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code} - {response.text}")
    
    data = response.json()
    properties = data.get("value", [])
    print(f"✓ Fetched {len(properties)} properties from API")
    return properties

"""Populate database with properties"""

database_url = f"sqlite:///{os.path.join("", 'properties.db')}"



In [47]:
database_url

'sqlite:///properties.db'

In [ ]:
# Populate Database with Properties
# Run this cell after installing dependencies: pip install -r requirements.txt

import sys
import os


from scripts.populate_database import populate_database

# Populate database with first 500 properties
print("Starting database population...")
populate_database(limit=500)
print("\n✓ Database population complete!")


In [49]:
from services.property_transformer import transform_property, transform_media

engine = init_database(database_url)
session = get_session(engine)

for idx, raw_prop in enumerate(raw_properties[:5], 1):
    property_data = transform_property(raw_prop)
    print(property_data)

{'id': 'NWM1555351', 'listing_id': 'NWM1555351', 'listing_key': 'NWM107237936', 'list_price': 824900, 'street_number': '21717', 'street_name': '33rd', 'city': 'Sammamish', 'state_or_province': 'WA', 'postal_code': '98075', 'unparsed_address': '21717 SE 33rd Place , Sammamish, WA 98075-6267', 'property_type': 'Residential', 'property_sub_type': 'Residential', 'bedrooms_total': 4, 'bathrooms_total_integer': 3, 'bathrooms_full': 1, 'bathrooms_half': 1, 'living_area': 2310, 'lot_size_square_feet': 14000, 'year_built': 1977, 'standard_status': 'Closed', 'latitude': 47.5792, 'longitude': -122.048816, 'public_remarks': 'An immaculate home awaits in the heart of Sammamish! This warm & welcoming 4 bd, 2.25 bth home boasts a tradition layout w/spacious living & dining rooms sporting vaulted ceilings & skylights. Large family rm offers bright & open floorplan w/wall of windows overlooking the park-like, fully fenced backyard w/water feature. Spacious kitchen offers stainless steel appliances & ea

In [50]:
session.query(Property).filter_by(id=property_data["id"]).first()

In [ ]:
# Create property object
property_obj = Property(**property_data)
session.add(property_obj)